# Training a DQN with social attention on `intersection-v0`



In [ ]:
#@title Import requirements

# Environment
%pip install highway-env
import gymnasium as gym

# Agent
%pip install git+https://github.com/eleurent/rl-agents#egg=rl-agents
#from stable_baselines3 import PPO
#%load_ext tensorboard

# Visualisation utils
%pip install moviepy #env = gymnasium.make("highway-v0")

%pip install imageio_ffmpeg
import sys
!%pip install tensorboardx gym pyvirtualdisplay
# !%apt-get install -y xvfb ffmpeg
!%git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
sys.path.insert(0, '/HighwayEnv/scripts/') # -----------------------------------------------------------------
# from HighwayEnv.scripts.utils import show_videos
import warnings
warnings.filterwarnings("ignore")

## Training

We use a policy architecture based on social attention, see [[Leurent and Mercat, 2019]](https://arxiv.org/abs/1911.12250).


In [ ]:
buffer = []   # raccoglie da evaluation.rewards
data = []     # raccoglie i gruppi da 5 da buffer
std_devs = []

n_episodes = 200

In [ ]:
from rl_agents.trainer.evaluation import Evaluation
import time
import numpy

import logging

logger = logging.getLogger(__name__)

class CustomEvaluation(Evaluation):
  rewards = []

  def run_episodes(self):
    for self.episode in range(self.num_episodes):
      # Run episode
      terminal = False
      self.reset(seed=self.episode)
      rewards = []
      start_time = time.time()
      while not terminal:
        # Step until a terminal step is reached
        reward, terminal = self.step()
        rewards.append(reward)

          # Catch interruptions
        try:
          if self.env.unwrapped.done:
            break
        except AttributeError:
          pass

      # End of episode
      duration = time.time() - start_time
      self.after_all_episodes(self.episode, rewards, duration)
      self.after_some_episodes(self.episode, rewards)
      if self.episode%10 == 0:
        #stoppare il training
        
        pass

  def train(self):
    self.training = True
    if getattr(self.agent, "batched", False): 
        self.run_batched_episodes()
    else:
        self.run_episodes()
    self.close()
      
  def after_all_episodes(self, episode, rewards, duration):
    rewards = numpy.array(rewards)
    gamma = self.agent.config.get("gamma", 1)
    self.writer.add_scalar('episode/length', len(rewards), episode)
    self.writer.add_scalar('episode/total_reward', sum(rewards), episode)
    self.writer.add_scalar('episode/return', sum(r*gamma**t for t, r in enumerate(rewards)), episode)
    self.rewards.append(sum(r*gamma**t for t, r in enumerate(rewards)))
    self.writer.add_scalar('episode/fps', len(rewards) / max(duration, 1e-6), episode)
    self.writer.add_histogram('episode/rewards', rewards, episode)
    logger.info("Episode {} score: {:.1f}".format(episode, sum(rewards)))

  def step(self):
      """
          Plan a sequence of actions according to the agent policy, and step the environment accordingly.
      """
      # Query agent for actions sequence
      actions = self.agent.plan(self.observation)

      # Forward the actions to the environment viewer
      try:
          self.env.unwrapped.viewer.set_agent_action_sequence(actions)
      except AttributeError:
          pass

      # Step the environment
      previous_observation, action = self.observation, actions[0]
      transition = self.wrapped_env.step(action)
      self.observation, reward, done, truncated, info = transition
      terminal = done or truncated

      # Call callback
      if self.step_callback_fn is not None:
          self.step_callback_fn(self.episode, self.wrapped_env, self.agent, transition, self.writer)

      # Record the experience.
      try:
          self.agent.record(previous_observation, action, reward, self.observation, done, info)
      except NotImplementedError:
          pass

     # Writing actions on a file  ----------------------------------------
      with open("actions.txt", "a") as file:
        file.write(str(action) + " ")

      with open("output.txt", "a") as file:  
        for obs in self.observation:
          for o in obs:
            #file.write("| " + str(i)+ ") Coordinates: (" + str(obs[1]) + ", " + str(obs[2]) + ") | direction: (" + str(round(obs[3],1)) + ", " + str(round(obs[4], 1)) + ") | Seno: " + str(obs[5]) + "; Coseno: " +  str(obs[6]) +"\n")
            file.write(str(round(o, 4)) + ' ')
          file.write('\n')

      if not actions:
          raise Exception("The agent did not plan any action")
      
      return reward, terminal
  
  def after_all_episodes(self, episode, rewards, duration):
    rewards = numpy.array(rewards)
    gamma = self.agent.config.get("gamma", 1)
    self.writer.add_scalar('episode/length', len(rewards), episode)
    self.writer.add_scalar('episode/total_reward', sum(rewards), episode)
    self.writer.add_scalar('episode/return', sum(r*gamma**t for t, r in enumerate(rewards)), episode)
    self.writer.add_scalar('episode/fps', len(rewards) / max(duration, 1e-6), episode)
    self.writer.add_histogram('episode/rewards', rewards, episode)
    logger.info("Episode {} score: {:.1f}".format(episode, sum(rewards)))


    with open("actions.txt", "a") as file:
      file.write("\n")
    with open("output.txt", "a") as file:
      file.write("\n") 



Run tensorboard locally to visualize training.

In [ ]:
#@title Prepare environment, agent, and evaluation process.

NUM_EPISODES = n_episodes  #@param {type: "integer"}


from rl_agents.agents.common.factory import load_agent, load_environment

# Get the environment and agent configurations from the rl-agents repository
!%git clone https://github.com/eleurent/rl-agents.git 2> /dev/null
%cd rl-agents/scripts/
env_config = 'configs/IntersectionEnv/env.json'
agent_config = 'configs/IntersectionEnv/agents/DQNAgent/ego_attention_2h.json'

env = load_environment(env_config)
agent = load_agent(agent_config, env)
evaluation = CustomEvaluation(env, agent, num_episodes=NUM_EPISODES, display_env=False, display_agent=False, sim_seed=237)
print(f"Ready to train {agent} on {env}")

Start training. This should take about an hour.

In [ ]:
#%cd ../tt/rl-agents/scripts/
with open("output.txt", "w") as file:
     pass
with open("actions.txt", "w") as file:
     pass

evaluation.train()
for d in evaluation.rewards:
  buffer.append(d)

model = evaluation.load_agent_model("final")
model

In [ ]:
import numpy
obs_data = numpy.loadtxt("output.txt")
act_data = numpy.zeros(n_episodes)
act_n = numpy.zeros(n_episodes)
context_data = [[], []] #[[0][n_ep][numero azione] , [1][n_ep][macchina][informazione]
block = 0

with open("actions.txt", "r") as file:
    for r, row in enumerate(file):
        matrix_list = []
        actions_list = []
        for e, el in enumerate(row):
            matrix=numpy.zeros((15,7))
            if not el == ' ' and not el == '\n':
                actions_list.append(el)
                for c in range(15):
                    for i in range(7):
                        #print(obs_data[c+15*block])
                        matrix[c][i] = obs_data[c+15*block][i] 
                matrix_list.append(matrix)   
                block +=1
        context_data[0].append(numpy.asarray(actions_list))
        context_data[1].append(numpy.asarray(matrix_list))



In [ ]:
def lanes_finder(car):                  #cell 3 = vx; cell 4 = vy
    if car[1] < 0 and car[2] <0: 
        # upper left
        if car[3] == 0:                      #| if abs(car[4]) > abs(car[3]):
            return "3,1"
        elif car[3] < 0:                     #| if abs(car[3]) >= abs(car[4]):
            return "2,2"
        elif car[4] < 0:
            return "2,3"  
        else:                           
            return "3,4"

    elif car[1] >= 0 and car[2] <0:
        # upper right
        if car[4] == 0:
            return "4,2"
        elif car[4] < 0:
            return "3,3"
        elif car[3] > 0:
            return "3,4"  
        else:
            return "4,1"
        
    elif car[1] >=0 and car[2] >=0:
        # lower right
        if car[3] == 0:
            return "1,3"
        elif car[3] > 0:
            return "4,4"
        elif car[4] > 0:
            return "4,1"  
        else:
            return "1,2"

    else:
        # lower left
        if car[4] == 0:
            return "2,4"
        elif car[4] > 0:
            return "1,1"
        elif car[3] < 0:
            return "1,2"  #
        else:
            return "2,3" 


In [ ]:
with open("faster2.las", "w") as file: #TODO crea una cartella in cui conservare i batch, se non direttamente qui
    file.write("limit(1..120).\nlane(1..4).\ndistance(1..40).\n\n")

    for ep in range(n_episodes): #uno dei 4 episodi
        for a, action in enumerate(context_data[0][ep]):  #una delle azioni             
            ex_string = "#pos(id_ep" + str(ep) + "_act"+str(a)+"@1, {"
            if action == '0':             #slower
                ex_string += "},{faster"
            elif action == '1':           #idle
                ex_string += "},{faster"
            elif action == '2':           #faster
                ex_string += "faster},{"
            ex_string += "}, {ego(" + lanes_finder(context_data[1][ep][a][0]) + ")."

            existing_cars = []
            for car in context_data[1][ep][a][1:]:
                if car[0] == 1:  # car is present
                    existing_cars.append(numpy.asarray(car))

            distances = numpy.zeros((len(existing_cars))) # distanze di ogni ostacolo dall'agente
            if len(existing_cars) == 0:
                pass
            else:
                for c in range(len(existing_cars)):
                    distances[c] = int(numpy.sqrt((context_data[1][ep][a][0][1] - existing_cars[c][1])**2 + (context_data[1][ep][a][0][2] - existing_cars[c][2])**2)*100)
                ex_string += " obs_car(" + str(min(distances)) + ","
                min_index = numpy.argmin(distances)   

                ex_string += lanes_finder(existing_cars[min_index]) + ")."
            ex_string += "})."
                
            file.write(ex_string + "\n")
            ex_string = ""

    file.write('\nobs_is_far(Dist, Dl) :- obs_car(Dist, L1, L2), limit(Dl), Dist > Dl. \nobs_is_crossing :- obs_car(Dist, L1, L2), L1 != L2. \nsame_lane :- ego(L1a, L2a), obs_car(Dist, L1, L2), L1a = L1, L2a = L2.\n\n')
    file.write("\nmodeh(faster). \n#modeb(1, obs_is_far(var(distance),  const(limit))). \n#modeb(1, obs_is_crossing). \n#modeb(1, same_lane). \n#modeb(not same_lane). \n#maxv(4). \n\n")
    file.write("#bias(" + '"' + "penalty(1, head(X)) :- in_head(X)." +'"' + ").")

In [ ]:
import numpy as np

print(len(evaluation.rewards))
print(len(buffer))


print(numpy.mean(evaluation.rewards), np.std(evaluation.rewards))

In [ ]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd

for i in range(NUM_EPISODES):  #Mi salvo in ogni cella, gli i-esimi valori del buffer, messi in append, così da avere un array con ogni cella occupata da una quintupla di valori
    group = [
        buffer[i],
        buffer[i + NUM_EPISODES],
        buffer[i + 2*NUM_EPISODES],
        buffer[i + 3*NUM_EPISODES],
        buffer[i + 4*NUM_EPISODES]
    ]
    data.append(np.mean(group)) # i data sono i mean return
    std_devs.append(np.std(group)) #array di deviazioni standard

# Calcolo la media e la deviazione standard totali
mean = np.mean(data)
dev = np.std(data)

print(mean, dev)

#Calcolo media sul valore bilanciato
stabilizer = 4500
data = []
std_devs = []
for i in range(NUM_EPISODES-stabilizer):
    group = [
        buffer[stabilizer+i],
        buffer[stabilizer + i + NUM_EPISODES],
        buffer[stabilizer + i + NUM_EPISODES],
        buffer[stabilizer + i + NUM_EPISODES],
        buffer[stabilizer + i + NUM_EPISODES]
    ]
    data.append(np.mean(group)) # i data sono i mean return
    std_devs.append(np.std(group)) #array di deviazioni standard

mean = np.mean(data)
dev = np.std(data)
print(mean, dev)

# Grafico
print("Grafico originale")
plot.figure(figsize=(16, 6))

plot.plot(data, label='Return')
plot.fill_between(range(len(data)),
                 np.array(data) - np.array(std_devs),
                 np.array(data) + np.array(std_devs),
                 color='b', alpha=0.2, label='Std Dev')
plot.xlabel('Episodes')
plot.ylabel('Return')
plot.title('Return vs Episodes')
plot.legend()
plot.show()

# ---------------------------------------------------------------
print("\n\nGrafico Smoothed\n\n")
window = 100

mean_smoothed = pd.Series(data).rolling(window=window).mean()
std_smoothed = pd.Series(data).rolling(window=window).std()


plot.figure(figsize=(16, 6))

plot.plot(mean_smoothed, label='Mean Return (Smoothed)', color='blue')
plot.fill_between(range(len(mean_smoothed)),
                mean_smoothed - std_smoothed,
                mean_smoothed + std_smoothed,
                color='blue', alpha=0.2, label='Std Dev (Smoothed)')
plot.xlabel('Episodes')
plot.ylabel('Return')
plot.title('Return vs Episodes')
plot.legend()
plot.show()

data.clear()
std_devs.clear()



## Testing

In [ ]:
v_test = []
print(env_config)

In [ ]:
from rl_agents.agents.common.factory import load_agent, load_environment
from rl_agents.agents.deep_q_network.graphics import *

#@title Run the learned policy for a few episodes.
print(env_config)
env = load_environment(env_config)
env.env.env.config['offscreen_rendering'] = True
# env.config["offscreen_rendering"] = True    
agent = load_agent(agent_config, env)
evaluation = CustomEvaluation(env, agent, num_episodes=100, training=False, recover=True, sim_seed=237)
evaluation.test()

# show_videos(evaluation.run_directory)

In [ ]:
print(len(evaluation.rewards))

In [ ]:
for val in evaluation.rewards[_:len(evaluation.rewards)]:
  v_test.append(val)

print(len(v_test))

In [ ]:
st_dev=numpy.std(v_test)
mean_reward=numpy.mean(v_test)


print(f"Return Medio: {mean_reward}; Standard Deviation: {st_dev}")

# Grafico
plot.figure(figsize=(16, 6))

plot.plot(v_test, label='Return')
plot.fill_between(range(len(v_test)),
                 np.array(v_test) - st_dev,
                 np.array(v_test) + st_dev,
                 color='g', alpha=0.3, label='Std Dev')
plot.xlabel('Episodes')
plot.ylabel('Return')
plot.title('Testing Results')
plot.show()

# ---------------------------------------
print("\n\nGrafico Smoothed\n\n")
window = 100

mean_smoothed = pd.Series(v_test).rolling(window=window).mean()
std_smoothed = pd.Series(v_test).rolling(window=window).std()


plot.figure(figsize=(16, 6))

plot.plot(mean_smoothed, label='Mean Return (Smoothed)', color='blue')
plot.fill_between(range(len(mean_smoothed)),
                mean_smoothed - std_smoothed,
                mean_smoothed + std_smoothed,
                color='green', alpha=0.2, label='Std Dev (Smoothed)')
plot.xlabel('Episodes')
plot.ylabel('Return')
plot.title('Return vs Episodes')
plot.legend()
plot.show()
